# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [29]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine

In [30]:
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package punkt to /home/apu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/apu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/apu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [31]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', engine)
#df.head()
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)

In [32]:
#Y

### 2. Write a tokenization function to process your text data

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [34]:
def tokenize(text):
    # Define url pattern
    url_re = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Detect and replace urls
    detected_urls = re.findall(url_re, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # tokenize sentences
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # save cleaned tokens
    clean_tokens = [lemmatizer.lemmatize(tok).lower().strip() for tok in tokens]
    
    # remove stopwords
    STOPWORDS = list(set(stopwords.words('english')))
    clean_tokens = [token for token in clean_tokens if token not in STOPWORDS]
    
    return clean_tokens

In [28]:
a=tokenize("It is a far, far better thing that I do, than I have ever done; it is a far, far better rest I go to than I have ever known.")
b=CountVectorizer(a)
c = b.fit_transform(a)
c

/home/apu/anaconda3/envs/udacity/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass input=['far', ',', 'far', 'better', 'thing', ',', 'ever', 'done', ';', 'far', ',', 'far', 'better', 'rest', 'go', 'ever', 'known', '.'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


<18x8 sparse matrix of type '<class 'numpy.int64'>'
	with 13 stored elements in Compressed Sparse Row format>

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [35]:
def build_pipeline():
    
    # build NLP pipeline - count words, tf-idf, multiple output classifier
    pipeline = Pipeline([
        ('vec', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 100, n_jobs = -1)))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = build_pipeline()
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vec',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f270a5cc8c0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
def report(pipeline, X_test, Y_test):
    # predict on the X_test
    Y_pred = pipeline.predict(X_test)
    
    # build classification report on every column
    performances = []
    for i in range(len(Y_test.columns)):
        performances.append([f1_score(Y_test.iloc[:, i].values, Y_pred[:, i], average='micro'),
                             precision_score(Y_test.iloc[:, i].values, Y_pred[:, i], average='micro'),
                             recall_score(Y_test.iloc[:, i].values, Y_pred[:, i], average='micro')])
    # build dataframe
    performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = Y_test.columns)   
    return performances

report(pipeline, X_test, y_test)

,f1 score,precision,recall
related,0.806683,0.806683,0.806683
request,0.901892,0.901892,0.901892
offer,0.996338,0.996338,0.996338
aid_related,0.776472,0.776472,0.776472
medical_help,0.931034,0.931034,0.931034
medical_products,0.955142,0.955142,0.955142
search_and_rescue,0.972536,0.972536,0.972536
security,0.981691,0.981691,0.981691
military,0.966433,0.966433,0.966433
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
pipeline = build_pipeline()

In [37]:

from sklearn.model_selection import GridSearchCV
parameters = {
    'clf__estimator__n_estimators':[10,50,100]
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs= -1)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vec',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f26e2989dd0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=-1)))]),
             n_jobs=-1,
             param_grid={'clf__estimator__n_estimators': [10, 50, 100]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
cv.best_params_

{'clf__estimator__n_estimators': 50}

In [39]:
report(cv, X_test, y_test)

,f1 score,precision,recall
related,0.802868,0.802868,0.802868
request,0.897162,0.897162,0.897162
offer,0.995880,0.995880,0.995880
aid_related,0.778303,0.778303,0.778303
medical_help,0.924474,0.924474,0.924474
medical_products,0.957278,0.957278,0.957278
search_and_rescue,0.971468,0.971468,0.971468
security,0.983674,0.983674,0.983674
military,0.967043,0.967043,0.967043
child_alone,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [16]:
pipeline_improved = Pipeline([
                                ('vect', CountVectorizer(tokenizer=tokenize)),
                                ('tfidf', TfidfTransformer()),
                                ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators = 100)))
                            ])
pipeline_improved.fit(X_train, y_train)
y_pred_improved = pipeline_improved.predict(X_test)

In [17]:
report(pipeline_improved, X_test, y_test)

,f1 score,precision,recall
related,0.763961,0.763961,0.763961
request,0.890143,0.890143,0.890143
offer,0.993897,0.993897,0.993897
aid_related,0.766555,0.766555,0.766555
medical_help,0.928288,0.928288,0.928288
medical_products,0.956515,0.956515,0.956515
search_and_rescue,0.969789,0.969789,0.969789
security,0.979707,0.979707,0.979707
military,0.971315,0.971315,0.971315
child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [41]:
import pickle

pickle.dump(pipeline, open('classifier.pkl', 'wb'))


#pickle.dump(pipeline_improved, open('adaboost_model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.